# 外れ値(outlier)

他から大きく外れた値があると、予測結果がその値に左右されることがある。

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

X = np.arange(6)[:, np.newaxis]
y = - X.flatten()
X[-1] = 4
y[-1] = 4

xx = np.array([X.min() - .5, X.max() + .5])[:, np.newaxis]
yy = LinearRegression().fit(X, y).predict(xx)

plt.title('Regression with Outlier')

plt.scatter(X, y)
plt.plot(xx, yy)

plt.xlim(xx.min(), xx.max())
plt.xticks(())
plt.yticks(())

plt.show()

外れ値(outlier)の問題は、異常値検出(anomaly detection)と重なる部分が大きい。

## 高次元の外れ値<a name="high_dimention"></a>

低次元では外れ値に見えても他の特徴と合わせた高次元では(結果に悪影響を与える)外れ値とは言えない場合や、その逆もある。

In [ ]:
n_sample = 100
n_out = 10
n_out_half = n_out // 2
margin = 3

np.random.seed(0)
X = np.empty((2, 2, n_sample))
X[0, 0] = np.random.uniform(low=0, high=8, size=n_sample)
tmp = np.random.uniform(low=11, high=12, size=n_out)
X[0, 0, : n_out_half] = tmp[: n_out_half]
X[0, 0, - n_out_half :] = tmp[- n_out_half :]
X[1, 0] = np.sort(np.random.normal(loc=0, scale=3, size=n_sample))
X[:, 1] = X[:, 0] + np.random.normal(size=X[:, 0].shape)
X[1, 1, : n_out_half] = X[1, 0, - n_out_half:]
X[1, 1, - n_out_half:] = X[1, 0, : n_out_half]

fig, axes = plt.subplots(2, 2, figsize=(8, 8))

for row, axes_in_row in enumerate(axes):
    for col, ax in enumerate(axes_in_row):
        x1 = X[row, 0]
        x2 = X[row, 1]

        include_outlier = (row + col) % 2 == 0

        title = '' if include_outlier else 'not '
        title += 'likely to be Outlier'
        ax.set_title(title)

        id_in = np.arange(n_out_half, n_sample - n_out_half, dtype=np.uint)
        id_out = np.concatenate((np.arange(n_out_half, dtype=np.uint), np.arange(n_out_half, dtype=np.uint) + n_sample - n_out_half))
        color = 'r' if include_outlier else 'b'
        if col == 0:
            r = (X[row, 0].min(), X[row, 0].max())

            if not include_outlier:
                ax.hist(x1, color=color, range=r)
            else:
                ax.hist(x1[id_in], color='b', range=r)
                ax.hist(x1[id_out], color=color, range=r)
        else:
            ax.scatter(x1[id_in], x2[id_in], color='b')
            ax.scatter(x1[id_out], x2[id_out], color=color)
            lin_min, lin_max = x1.min() - margin, x1.max() + margin

        ax.set_xticks(())
        ax.set_yticks(())

plt.show()

## 外れ値が生じる原因<a name="cause"></a>

外れ値が生じる原因はいろいろ考えられる。

- センサーの故障
- データ入力ミス
- 表計算ソフトの集計行混入
- 特異なイベントの発生(近くでイベントが行われた等)

外れ値の生じた原因に応じて、取り扱いを考える必要がある。外れ値が起こるべきでない異常によって生じた場合は外れ値をデータから取り除き、外れ値が予測を改善しそうな情報を含む場合はそれを捉える特徴を新たに設計することで、性能を向上させられる。

## 外れ値の除去<a name="removal"></a>

極端に誤差・残差の大きいものや、上位10%を除去する。やりすぎると過学習しやすくなるので慎重に行う。また、外れ値がセンサーの故障などで定期的に発生する可能性がある場合は、異常値として検出できるようにしておく。

In [ ]:
from sklearn.linear_model import LinearRegression

np.random.seed(0)
X = np.sort(np.random.uniform(low=0, high=10, size=100))
y = X + np.random.normal(size=X.shape)
y[:5], y[-5:] = X[-5:], X[:5]

regr = LinearRegression().fit(X[:, np.newaxis], y)
pred = regr.predict(X[:, np.newaxis])
residual = y - pred
id_out = np.argsort(np.abs(residual))[-10:]

print('Index of highest 10% residual: {res}'.format(res=id_out))

plt.figure(figsize=(4, 4))

plt.scatter(pred, residual, color='b')
plt.scatter(pred[id_out], residual[id_out], color='r')

plt.xticks(())
plt.yticks(())

plt.show()